# Step 2: Visualization

This notebook loads the interpolated phase space distribution and generates interactive 2D figures.

In [ ]:
import importlib
import itertools
import os
import sys
import time

from ipywidgets import interactive
from ipywidgets import widgets
import h5py
from matplotlib import pyplot as plt
import numpy as np
import proplot as pplt
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from scipy import ndimage

sys.path.append('/Users/46h/Research/btf-data-analysis/')
from tools import utils

sys.path.append('/Users/46h/Research/psdist/')
from psdist import image as psi
from psdist import plotting as mplt

## Settings 

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

In [ ]:
folder = './_output/'

## Load data 

In [ ]:
os.listdir(folder)

Load the phase space density array `f`.

In [ ]:
info = utils.load_pickle(os.path.join(folder, 'info.pkl'))
filename = info['filename']
shape = info['int_shape']
f = np.memmap(os.path.join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

print('f.shape:', shape)
print(f'f.nbytes = {f.nbytes:.4e}')

Apply a Gaussian filter with $\sigma = 0.7$ to `f` for a slight smoothing effect. (This takes a few minutes.)

In [ ]:
f = ndimage.gaussian_filter(f, 0.7)

Load the grid coordinates `coords`.

In [ ]:
coords = utils.load_stacked_arrays(os.path.join(folder, f'coords_{filename}.npz'))
dims = info['dims']
units = info['units']

# Convert MeV --> keV.
coords[4] = coords[4] * 1000.0
units[4] = 'keV'

dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]

print('dims:', info['dims'])
print('units:', info['units'])

Center the coordinates on the centroid of the distribution, as calculated from the 1D projections.

In [ ]:
profs = []
for i in range(5):
    prof = psi.project(f, axis=i)
    prof = prof / np.sum(prof)
    coords[i] = coords[i] - np.average(coords[i], weights=prof)
    profs.append(prof)

Crop the 5D image:

In [ ]:
crop = (
    (0, f.shape[0]),
    (10, f.shape[1] - 10),
    (0, f.shape[2]),
    (19, f.shape[3] - 16),
    (15, f.shape[4] - 20),
)

fig, axes = pplt.subplots(ncols=5, figwidth=7, spanx=False, figheight=None)
for i, ax in enumerate(axes):
    ax.plot(profs[i], color='black')
    ax.axvspan(crop[i][0], crop[i][1] - 1, color='black', alpha=0.1)
    ax.format(xlabel=dims[i] + ' [pixel]')
# axes.format(yscale='log')
plt.show()

In [ ]:
ind = tuple([slice(c[0], c[1]) for c in crop])
f = f[ind]
coords = [c[ind[i]] for i, c in enumerate(coords)]

Normalize `f` to the range [0, 1].

In [ ]:
f = f / np.max(f)
print(f'f_min = {np.min(f)}')
print(f'f_max = {np.max(f)}')

Apply a threshold to `f`.

In [ ]:
f[f < 10.0**-3.5] = 0

## 1D projections 

In [ ]:
mplt.interactive_proj1d(
    f, coords=coords, dims=dims, units=units, default_ind=4,
    kind='line',  # {'line', 'step', 'bar'}
    slice_type='int',  # {'int', 'range'}
)

## 2D projections 

In [ ]:
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12)
cmaps = ['viridis', 'mono_r', 'plasma', pplt.Colormap('mono', right=0.8), 'dusk_r']

The `thresh` slider applies a fractional threshold to the 2D projection; a good value is around -3.0 for these measurements.

In [ ]:
mplt.interactive_proj2d(
    f, coords=coords, dims=dims, units=units, 
    prof_kws=prof_kws,
    slice_type='int',  # {'int', 'range'}
    cmaps=cmaps,
)

In [ ]:
def update(cmap, log, diag):
    axes = mplt.corner(
        f, 
        coords=coords,
        diag_kind='step' if diag else 'None',
        prof=False if diag else 'edges',
        labels=dims_units,
        norm='log' if log else None,
        handle_log='floor',
        thresh=10.0**-3.5,
        thresh_type='frac',
        linewidth=0, rasterized=True,
        prof_kws=prof_kws,
        cmap=cmap,
    )
    return axes


interactive(update, cmap=cmaps, log=False, diag=True)